# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.36.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-176882
aml-quickstarts-176882
southcentralus
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-176882,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "divorce-ds"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key]
        print('dataset found in workspace')

if not found:
        print('dataset NOT found in workspace')
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/joangerard/project-azure-piepline/master/divorce_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data, separator=';')        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

dataset found in workspace


,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Divorce
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,...,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,1.776471,1.652941,1.764706,1.482353,1.541176,0.747059,0.494118,1.452941,1.458824,1.576471,...,2.552941,2.270588,2.741176,2.382353,2.429412,2.476471,2.517647,2.241176,2.011765,0.494118
std,1.627257,1.468654,1.415444,1.504327,1.632169,0.904046,0.898698,1.546371,1.557976,1.421529,...,1.371786,1.586841,1.137348,1.511587,1.405090,1.260238,1.476537,1.505634,1.667611,0.501442
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000
50%,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,...,3.000000,2.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000,2.000000,0.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,3.000000,3.000000,3.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [6]:
dataset.take(5).to_pandas_dataframe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Divorce
0,2,2,4,1,0,0,0,0,0,0,...,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,...,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,...,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,...,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,...,2,1,2,3,2,2,2,1,0,1


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Divorce",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [9]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [10]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [11]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [910d7fd7][c9f580b0-0ce7-4858-8cb1-a5df9e6d8905], (This step will run and generate new outputs)
Submitted PipelineRun b5080aa6-13e8-4bfb-852f-5c018019cec1
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b5080aa6-13e8-4bfb-852f-5c018019cec1?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-176882/workspaces/quick-starts-ws-176882&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [12]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
pipeline_run.wait_for_completion()

PipelineRunId: b5080aa6-13e8-4bfb-852f-5c018019cec1
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b5080aa6-13e8-4bfb-852f-5c018019cec1?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-176882/workspaces/quick-starts-ws-176882&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 8225b235-bbc9-4c45-907f-aadd86c28980
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/8225b235-bbc9-4c45-907f-aadd86c28980?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-176882/workspaces/quick-starts-ws-176882&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [14]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/8225b235-bbc9-4c45-907f-aadd86c28980/metrics_data, 1 files out of an estimated total of 1


In [15]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,8225b235-bbc9-4c45-907f-aadd86c28980_27,8225b235-bbc9-4c45-907f-aadd86c28980_32,8225b235-bbc9-4c45-907f-aadd86c28980_16,8225b235-bbc9-4c45-907f-aadd86c28980_26,8225b235-bbc9-4c45-907f-aadd86c28980_34,8225b235-bbc9-4c45-907f-aadd86c28980_31,8225b235-bbc9-4c45-907f-aadd86c28980_22,8225b235-bbc9-4c45-907f-aadd86c28980_29,8225b235-bbc9-4c45-907f-aadd86c28980_25,8225b235-bbc9-4c45-907f-aadd86c28980_30,...,8225b235-bbc9-4c45-907f-aadd86c28980_18,8225b235-bbc9-4c45-907f-aadd86c28980_8,8225b235-bbc9-4c45-907f-aadd86c28980_11,8225b235-bbc9-4c45-907f-aadd86c28980_10,8225b235-bbc9-4c45-907f-aadd86c28980_12,8225b235-bbc9-4c45-907f-aadd86c28980_13,8225b235-bbc9-4c45-907f-aadd86c28980_19,8225b235-bbc9-4c45-907f-aadd86c28980_21,8225b235-bbc9-4c45-907f-aadd86c28980_23,8225b235-bbc9-4c45-907f-aadd86c28980_17
recall_score_macro,[0.9747222222222222],[0.9187698412698413],[0.9747222222222222],[0.9747222222222222],[0.9747222222222222],[0.9747222222222222],[0.9511111111111111],[0.9684722222222222],[0.9858333333333335],[0.9802777777777777],...,[0.9684722222222222],[0.9747222222222222],[0.9684722222222222],[0.9684722222222222],[0.9747222222222222],[0.9747222222222222],[0.9636111111111111],[0.9629166666666666],[0.9747222222222222],[0.9747222222222222]
recall_score_weighted,[0.9764705882352942],[0.9235294117647059],[0.9764705882352942],[0.9764705882352942],[0.9764705882352942],[0.9764705882352942],[0.9529411764705884],[0.9705882352941178],[0.9882352941176471],[0.9823529411764707],...,[0.9705882352941178],[0.9764705882352942],[0.9705882352941178],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9647058823529413],[0.9647058823529413],[0.9764705882352942],[0.9764705882352942]
average_precision_score_macro,[0.99875],[0.9616711752526703],[1.0],[1.0],[1.0],[0.99875],[0.9952553310886645],[0.9975000000000002],[1.0],[0.995203477078477],...,[0.99875],[1.0],[0.99875],[0.99875],[1.0],[1.0],[1.0],[0.99875],[1.0],[0.9980128205128205]
precision_score_macro,[0.9767094017094017],[0.9308923021423021],[0.9767094017094017],[0.9767094017094017],[0.9767094017094017],[0.9767094017094017],[0.9573785936285937],[0.9704594017094017],[0.9878205128205128],[0.9822649572649572],...,[0.9704594017094017],[0.9767094017094017],[0.9704594017094017],[0.9704594017094017],[0.9767094017094017],[0.9767094017094017],[0.9686285936285935],[0.9660149572649572],[0.9767094017094017],[0.9767094017094017]
recall_score_micro,[0.9764705882352942],[0.9235294117647059],[0.9764705882352942],[0.9764705882352942],[0.9764705882352942],[0.9764705882352942],[0.9529411764705884],[0.9705882352941178],[0.9882352941176471],[0.9823529411764707],...,[0.9705882352941178],[0.9764705882352942],[0.9705882352941178],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9647058823529413],[0.9647058823529413],[0.9764705882352942],[0.9764705882352942]
f1_score_micro,[0.9764705882352942],[0.9235294117647059],[0.9764705882352942],[0.9764705882352942],[0.9764705882352942],[0.9764705882352942],[0.9529411764705884],[0.9705882352941178],[0.9882352941176471],[0.9823529411764707],...,[0.9705882352941178],[0.9764705882352942],[0.9705882352941178],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9647058823529413],[0.9647058823529413],[0.9764705882352942],[0.9764705882352942]
log_loss,[0.0740931194067871],[0.36693524364938923],[0.061556207339617766],[0.04796118582822523],[0.0637632951468783],[0.15385718581289248],[0.29889844694038553],[0.13775454109486338],[0.16215353599664167],[0.5026820034906467],...,[0.12424090632583987],[0.0702554035389648],[0.27928275608402325],[0.14233186299666456],[0.08276036113345764],[0.041377260922129795],[0.15471303379051465],[0.10119714890899575],[0.07451981468445137],[0.07453153105467879]
AUC_macro,[0.9986111111111111],[0.9730357142857142],[1.0],[1.0],[1.0],[0.9986111111111111],[0.9944444444444445],[0.9972222222222221],[1.0],[0.9949999999999999],...,[0.9986111111111111],[1.0],[0.9986111111111111],[0.9986111111111111],[1.0],[1.0],[1.0],[0.9986111111111111],[1.0

### Retrieve the Best Model

In [16]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/8225b235-bbc9-4c45-907f-aadd86c28980/model_data, 1 files out of an estimated total of 1


In [17]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                                        class_weight='balanced',
                                        criterion='entropy', max_depth=None,
                                        max_features=0.1, max_leaf_nodes=None,
                                        max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=0.1131578947368421,
                                        min_samples_split=0.47842105263157897,
                                        m

In [18]:
best_model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('MaxAbsScaler', MaxAbsScaler(copy=True)),
 ('RandomForestClassifier',
  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                         criterion='entropy', max_depth=None, max_features=0.1,
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=0.1131578947368421,
                         min_samples_split=0.47842105263157897,
                         min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
                         oob_score=True, random_state=None, verbose=0,
               

In [34]:
# Register it
from azureml.core.run import Run

run = Run.get_context()
pipeline_run.upload_file("./outputs/model.pkl", best_model_output._path_on_datastore)
print(pipeline_run.get_file_names())
model = pipeline_run.register_model(model_name="auto-ml-model", model_path="./outputs/model.pkl")

['logs/azureml/executionlogs.txt', 'logs/azureml/stderrlogs.txt', 'logs/azureml/stdoutlogs.txt', 'outputs/model.pkl']
